In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [10]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
print(num_train)
print(height)
print(width)
print(depth)
print(X_train.shape)
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes
print(y_train.shape)

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

print(y_train)
Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
print(Y_train)
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels
print(Y_train.shape)

50000
32
32
3
(50000, 32, 32, 3)
(50000, 1)
[[6]
 [9]
 [9]
 ..., 
 [9]
 [1]
 [1]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]
(50000, 10)


In [ ]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

Train on 45000 samples, validate on 5000 samples
Epoch 1/200

 3008/45000 [=>............................] - ETA: 7:01 - loss: 2.1890 - acc: 0.1832


 6048/45000 [===>..........................] - ETA: 6:37 - loss: 2.0494 - acc: 0.2369


 9088/45000 [=====>........................] - ETA: 5:47 - loss: 1.9638 - acc: 0.2680


12128/45000 [=======>......................] - ETA: 5:09 - loss: 1.8970 - acc: 0.2931


15168/45000 [=========>....................] - ETA: 4:44 - loss: 1.8386 - acc: 0.3174


...

45000/45000 [==============================] - ETA: 0s - loss: 0.3887 - acc: 0.8635 - val_loss: 0.6795 - val_acc: 0.7962
Epoch 31/200

 2272/45000 [>.............................] - ETA: 6:50 - loss: 0.3746 - acc: 0.8684


 5312/45000 [==>...........................] - ETA: 6:24 - loss: 0.3682 - acc: 0.8682


 8352/45000 [====>.........................] - ETA: 5:56 - loss: 0.3652 - acc: 0.8695


11392/45000 [======>.......................] - ETA: 5:26 - loss: 0.3687 - acc: 0.8684


13984/45000 [========>.....................] - ETA: 5:06 - loss: 0.3750 - acc: 0.8691